# Unbalanced two-way ANOVA

This page follows on from the [two-way balanced ANOVA
notebook](./twoway_balanced.ipynb).

Please make sure you follow the two-way balanced ANOVA notebook before you read
this notebook, because we are going to re-use notation and machinery
from that notebook.

That notebook introduced the two-way Analysis of Variance (ANOVA).  It
generalizes the one-way ANOVA by looking at the means for the levels of the
different factors, and the means for each of the (here) six groups.

The two-way balanced ANOVA notebook introduced a cheat to simplify the
calculations, where we dropped some rows in order to make the number of rows in
each subgroup the same.  A sub-group is a group with the same level in the two
factors.  When there are two factors, one with 2 levels, the other with 3, then
there are 2 * 3 = 6 sub-groups.  In our example, one sub-group would be those
rows for which the `gender` Factor has level `Female` and the `diet` factor has
level `A`.

This notebook covers the more general and typical case, where there are not the same number in each sub-group.

As we will see, this introduces some complications that can only be solved by using multiple regression, and here, it is simpler to take the Extra Sum of Squares view of the SNSQGMD metric.  See the [oneway ANOVA notebook](./oneway_diet.ipynb) for a longer explanation of the different ways of thinking of the SNSQGMD metric.


## Back again to the example


In [1]:
# Array, data frame and plotting libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Statsmodels ANOVA machinery.
import statsmodels.api as sm
import statsmodels.formula.api as smf

We return again to our dataset giving amount of weight lost (in kilograms)
after 10 weeks of one of three possible diets.

As in the one-way ANOVA page, we use the `gender` column to define one factor,
and the `diet` column to define another, so we can classify the rows
(individuals) into `Female` or `Male` (`gender`) and into `A`, `B` and `C`
(`diet`).

See: [the dataset page](https://github.com/odsti/datasets/tree/master/sheffield_diet) for more detail.

In [2]:
# Read the dataset
diet_data = pd.read_csv('sheffield_diet.csv')
diet_data.head()

,gender,diet,weight_lost
0,Female,A,3.8
1,Female,A,6.0
2,Female,A,0.7
3,Female,A,2.9
4,Female,A,2.8


Pandas `groupby` can classify each row using *both* the `gender` label (level) and the `diet` label (level).  Notice we have different number is the six possible sub-groups.

In [3]:
grouped = diet_data.groupby(['gender', 'diet'])
# Show the counts in each of the six groups.
grouped['weight_lost'].count()

gender  diet
Female  A       14
        B       14
        C       15
Male    A       10
        B       11
        C       12
Name: weight_lost, dtype: int64

## Unbalanced two-way ANOVA

Here is the standard two-way ANOVA, using Statsmodels.

In [4]:
unbal_fit = smf.ols('weight_lost ~ gender * diet', data=diet_data).fit()
# Type II (2) sum of squares calculation ANOVA table.
sm.stats.anova_lm(unbal_fit, typ=2)

,sum_sq,df,F,PR(>F)
gender,0.168696,1.0,0.031379,0.859910
diet,60.417220,2.0,5.619026,0.005456
gender:diet,33.904068,2.0,3.153204,0.048842
Residual,376.329043,70.0,NaN,NaN


Please keep referring back to this table.  This page will work through the
extra-sum-of-squares calculations to replicate the values in this table.


## Removing the overall mean

First we subtract the overall mean, and work on those subtracted values:

In [5]:
# "zm" stands for "zero mean"
diet_data_zm = pd.DataFrame()
diet_data_zm = diet_data.loc[:, :'diet'].copy()
weight_col = diet_data['weight_lost']
diet_data_zm['wlost_zm'] = weight_col - np.mean(weight_col)
diet_data_zm.head()

,gender,diet,wlost_zm
0,Female,A,-0.146053
1,Female,A,2.053947
2,Female,A,-3.246053
3,Female,A,-1.046053
4,Female,A,-1.146053


The overall mean of the ZM column is (near as dammit) zero.

In [6]:
np.mean(diet_data_zm['wlost_zm'])

-1.3264243504732134e-15

## In the unbalanced case, the group means depend on each other.

You may remember from the balanced ANOVA case that we wanted to remove the
effect of `diet` on the data, before looking at the SNSQGMD metric for
`gender`.  We found that, in fact, this didn't make any difference because subtracting the means for `diet` left the means for `gender` unchanged.

Now we are in the unbalanced case, that is not true.

Here are the means for the two `gender` levels without subtracting the means for `diet`:

In [7]:
diet_data_zm.groupby('gender')['wlost_zm'].mean()

gender
Female   -0.053029
Male      0.069099
Name: wlost_zm, dtype: float64

In the balanced case, we subtracted the means for `diet`, and got the same means for `gender`.

We will use the `subtract_means` function from the balanced ANOVA notebook.

In [8]:
# Put this into a function
def subtract_means(df, group_col, value_col):
    mean_col = df.groupby(group_col)[value_col].transform('mean')
    return df[value_col] - mean_col

When we subtract the diet mean, as expected, the mean for each diet group is near-as-dammit zero:

In [9]:
# Subtract diet mean.
wl_adj_diet = subtract_means(diet_data_zm, 'diet', 'wlost_zm')
# Insert this Series into new data frame.
adj_for_diet = diet_data_zm.assign(wl_adj_diet=wl_adj_diet)
# Mean for each diet group
adj_for_diet.groupby('diet')['wl_adj_diet'].mean()

diet
A    3.238150e-17
B    1.065814e-16
C   -1.069104e-16
Name: wl_adj_diet, dtype: float64

For the balanced case, we subtracted the `diet` and `gender` means before looking at the interaction.  We could subtract `diet` and then `gender` and not worry about the order in which we did this, because we would get the same answer from subtracting `gender` and then `diet`.  That is not true any more.

Above, we have adjusted for `diet`, but we have not yet adjusted for `gender` - so the means for gender are not near zero:

In [10]:
adj_for_diet.groupby('gender')['wl_adj_diet'].mean()

gender
Female   -0.041261
Male      0.053764
Name: wl_adj_diet, dtype: float64

Now we adjust for `gender`:

In [11]:
diet_then_gender = subtract_means(adj_for_diet, 'gender', 'wl_adj_diet')
diet_then_gender.head()

0    0.541261
1    2.741261
2   -2.558739
3   -0.358739
4   -0.458739
Name: wl_adj_diet, dtype: float64

Sure enough, the means are now near zero for `gender`, but now the `diet` means aren't very close to zero any more:

In [12]:
adj_for_d_then_g = adj_for_diet.assign(wl_adj_d_then_g=diet_then_gender)
adj_for_d_then_g.groupby('gender')['wl_adj_d_then_g'].mean()

gender
Female    1.068267e-16
Male     -1.110223e-16
Name: wl_adj_d_then_g, dtype: float64

In [13]:
adj_for_d_then_g.groupby('diet')['wl_adj_d_then_g'].mean()

diet
A    0.001667
B   -0.000550
C   -0.000972
Name: wl_adj_d_then_g, dtype: float64

If we subtract means for `gender`, then `diet`, we get the opposite effect, where the means for `diet` are very near zero, but the means for `gender` are not:

In [14]:
# Gender then diet.
wl_adj_gender = subtract_means(diet_data_zm, 'gender', 'wlost_zm')
adj_for_gender = diet_data_zm.assign(wl_adj_gender=wl_adj_gender)
gender_then_diet = subtract_means(adj_for_gender, 'diet', 'wl_adj_gender')
adj_for_g_then_d = adj_for_gender.assign(wl_adj_g_then_d=gender_then_diet)
# Diet means very close to 0
adj_for_g_then_d.groupby('diet')['wl_adj_g_then_d'].mean()

diet
A    4.625929e-17
B   -1.065814e-16
C    2.713879e-16
Name: wl_adj_g_then_d, dtype: float64

In [15]:
# Gender means now not very close to 0
adj_for_g_then_d.groupby('gender')['wl_adj_g_then_d'].mean()

gender
Female    0.011737
Male     -0.015294
Name: wl_adj_g_then_d, dtype: float64

When we are looking at the interaction effects, we need the means for `gender` and `diet` to be very near zero, so we cannot do the sequential steps we did before.  We need to find some way of *simultaneously* adjusting for `gender` and `diet`.

That is a job for *multiple regression*.  But, at the moment, we do not have our typical continuous numerical regressors for a multiple regression, we have columns with categories.  The next step is to get the right regressors for multiple regression to do this simultaneous adjustment.

## Means and multiple regression

In simple and multiple regression, we use one or more columns of *regressors* to *predict* another column of values.

Here we are trying to predict the values in the `weight_lost` column of the data frame.  Call this column `y`:

In [16]:
# Column we want to predict.  We could call this the 'dependent' variable.
y = diet_data['weight_lost']

Call the number of rows in this column: `n`.  In our case, this is the number
of individuals, and the number of rows in the individual data frame.

In [17]:
n = len(y)

Then we have other columns that we use to do the prediction.  These are *regressors*.

Now let us consider the following rather strange column of numbers:

In [18]:
x0 = pd.Series(np.ones(n))
x0

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
     ... 
71    1.0
72    1.0
73    1.0
74    1.0
75    1.0
Length: 76, dtype: float64

This is just a column of `n` ones.  In regression, we look for the best *parameters* to multiply the columns by, to give a good prediction for the data, `y`. We have one parameter per column.  In this case, we are going to make our prediction with some value, `b0`, yet to be determined.  For the moment, let's make `b0` be the completely arbitrary number 10:

In [19]:
b0 = 10

We are later going to find a better value for `b0`.

Our current prediction for `y` is formed by multiplying our regressor by the parameter:

In [20]:
prediction = b0 * x0
prediction

0     10.0
1     10.0
2     10.0
3     10.0
4     10.0
      ... 
71    10.0
72    10.0
73    10.0
74    10.0
75    10.0
Length: 76, dtype: float64

When we have a good prediction, we want the prediction to be as close as possible to the actual data `y`.   We assess that by subtracting the prediction from the actual values to give the *errors*:

In [21]:
errors = y - prediction
errors

0    -6.2
1    -4.0
2    -9.3
3    -7.1
4    -7.2
     ... 
71   -7.2
72   -5.9
73   -4.7
74   -0.8
75   -3.9
Length: 76, dtype: float64

We want to work out whether `b0` is a good parameter, so we want a score (metric) for `b0`.  We form the typical *least-squares* metric by squaring all the errors (to eliminate the sign) and adding them up.  Call this the *sum of squared error* or SSE.  We have a good `b0` when this number is small, and a bad one when this number is large.

In [22]:
SSE = np.sum(errors ** 2)
SSE

3256.35

We could try lots of `b0` values, and choose the best, but we will let Scipy's `minimize` do that for us:

In [23]:
# Function for Scipy minimize to use
def calc_sse(param, y, col):
    predicted = param * col
    errors = y - predicted
    return np.sum(errors ** 2)

In [24]:
# Check we get the same value as before
calc_sse(b0, y, x0)

3256.35

In [25]:
from scipy.optimize import minimize
res = minimize(calc_sse, 0, args=(y, x0))
res

      fun: 470.9288157894737
 hess_inv: array([[0.00657895]])
      jac: array([-3.81469727e-06])
  message: 'Optimization terminated successfully.'
     nfev: 8
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([3.94605261])

You can see from the `x.fun` value that `minimize` has found a much better value for the parameter `b0`.  The value it found was:

In [26]:
best_b0 = res.x
best_b0

array([3.94605261])

Here's the mean of the `y` values:

In [27]:
np.mean(y)

3.9460526315789486

That looks suspiciously similar - why?  Remember what we are doing in
`minimize`.  We are looking for the best value `b0` to multiply onto the
column, in order to predict `y`.  The column is all ones, so the prediction at
each row will be `b0`.  So the question becomes, what is the best single value
to subtract from `y` such that we minimize the resulting sum of squared
difference?  As you'll see in [the meaning of the
mean](https://matthew-brett.github.io/cfd2020/mean-slopes/mean_meaning.html),
the mean is exactly that value, so unless `minimize` messed up, it must find
the mean as the best value for `b0`.

Now let us think about finding two parameters, `b0` and `b1`, for two columns, `x0` and `x1`.

In fact we will use these columns to represent the group membership for the `gender` factor.

In [28]:
x_gender = pd.get_dummies(diet_data['gender'])
x_gender

,Female,Male
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
71,0,1
72,0,1
73,0,1
74,0,1


As you can see from Pandas' function name above, this is a *dummy coding* of the `gender` factor.  This type of dummy coding is also called *indicator* coding.  The first column — `Female` — has a 1 for rows corresponding to a `Female` label in the `gender` column, and a 0 otherwise.  A 1 value in this column *indicates* that this is a `Female` row.  Conversely the `Male` column has a 1 for rows with a `Male` label in the gender column, and a 0 otherwise.  The 1 *indicates* this is a `Male` row.

We will use these two columns in our regression.  Again, we are trying to find two parameters, one for each column.  The first parameter is the best number to multiply onto the first columns, and the second parameter corresponds to the second column.  The predicted value for each row is the result of adding (the first parameter times the corresponding first column value) to the (second parameter times the corresponding second column value).

Say we decided arbitrarily to use these parameters:

In [29]:
params = np.array([10, 5])

Then:

In [30]:
x_gender_arr = np.array(x_gender)
pred_from_col0 = params[0] * x_gender_arr[:, 0]
pred_from_col1 = params[1] * x_gender_arr[:, 1]
predicted =  pred_from_col0 + pred_from_col1

and:

In [31]:
errors = y - predicted
SSE = np.sum(errors ** 2)
SSE

2106.35

Make this into a function for `minimize`:

In [32]:
def calc_sse_for_cols(params, y, cols):
    cols_arr = np.array(cols)
    prediction = np.zeros(len(y))
    for i in np.arange(len(params)):
        prediction += params[i] * cols_arr[:, i]
    errors = y - prediction
    return np.sum(errors ** 2)

In [33]:
calc_sse_for_cols(params, y, x_gender)

2106.35

Find the best (least-squares) parameters for the two dummy columns:

In [34]:
res_gender = minimize(calc_sse_for_cols, [0, 0], args=(y, x_gender))
res_gender

      fun: 470.6503312191686
 hess_inv: array([[ 1.16279074e-02, -1.19430014e-09],
       [-1.19430014e-09,  1.51515185e-02]])
      jac: array([-3.81469727e-06, -3.81469727e-06])
  message: 'Optimization terminated successfully.'
     nfev: 24
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([3.89302324, 4.01515159])

How do these values compare to the means for each group?

In [35]:
diet_data.groupby('gender')['weight_lost'].mean()

gender
Female    3.893023
Male      4.015152
Name: weight_lost, dtype: float64

The same!  The least-squares parameters for the dummy columns are the respective means for the `Female` and `Male` values in `y`.  Why?

Remember the prediction is the sum of the predictions from the first and second
columns.  Once multiplied by the first parameter `params[0]`, the first column
will have the `params[0]` value where it had 1, and 0 otherwise.  Therefore,
the first column (`params[0]`) prediction is `params[0]` for the `Female` rows,
and 0 otherwise.  Conversely the second column (`params[1]`) prediction is
`params[1]` for the `Male` rows and 0 otherwise.  The `params[0]` should be the
best value to minimize the sum of squared error for the `Female` values, and
will have no effect in predicting the `Male` values.  `params[1]` should
minimize the SSE for the `Male` values, and will have no effect on the `Female`
prediction. This corresponds to two entirely separable predictions, and so
`params[0]` must be the mean of the `Female` values (to minimize the SSE), and
`params[1]` must be the mean of the `Male` values.

We are going to run a few of these least-square fits, and, for this specific
case of minimizing SSE, there is a mathematical short-cut to avoid using
`minimize`.  The details of how this works are not important here, but it
involves the calculus of mean squares that you will see sketched in [this
page](https://matthew-brett.github.io/cfd2020/extra/mean_sq_deviations.html).

Here is the shortcut:

In [36]:
# Mathematical short-cut to finding best SSE parameters
# We get the same answer from minimize, but this method is quicker # and more
# precise.
best_params = np.linalg.pinv(x_gender) @ y
best_params

array([3.89302326, 4.01515152])

In fact we can also write the multiplication of the parameters by the columns in a more efficient way, using Numpy broadcasting, like this:

In [37]:
predicted = np.sum(best_params * np.array(x_gender), axis=1)

In [38]:
errors = y - predicted
np.sum(errors ** 2)

470.6503312191684

Notice this gives us the same answer as the `minimize` fit for the best parameters, and the subsequent SSE calculation.

Here's all that packaged up into a function.

In [39]:
def calc_sse_shortcut(y, cols):
    params = np.linalg.pinv(cols) @ y
    predicted = np.sum(params * np.array(cols), axis=1)
    errors = y - predicted
    return np.sum(errors ** 2)

In [40]:
calc_sse_shortcut(y, x_gender)

470.6503312191684

We can do the same procedure to replicate the means for each diet:

In [41]:
x_diet = pd.get_dummies(diet_data['diet'])
x_diet

,A,B,C
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
71,0,0,1
72,0,0,1
73,0,0,1
74,0,0,1


In [42]:
diet_params = np.linalg.pinv(x_diet) @ y
diet_params

array([3.3       , 3.268     , 5.14814815])

In [43]:
diet_data.groupby('diet')['weight_lost'].mean()

diet
A    3.300000
B    3.268000
C    5.148148
Name: weight_lost, dtype: float64

In [44]:
predicted = np.sum(diet_params * np.array(x_diet), axis=1)
np.sum((y - predicted) ** 2)

410.40180740740743

In [45]:
calc_sse_shortcut(y, x_diet)

410.40180740740743

So far we haven't done anything very interesting, because we could get the same answer from getting the means for each group.  The interesting thing happens when we stack the columns for the two factors together.  When we do this, and find the least squares parameters, we are now finding the means for the `gender` levels *adjusting for diet* at the same times as the means for the `diet` levels *adjusting for gender*.

In [46]:
x_gender_diet = pd.concat([x_gender, x_diet], axis=1)
x_gender_diet

,Female,Male,A,B,C
0,1,0,1,0,0
1,1,0,1,0,0
2,1,0,1,0,0
3,1,0,1,0,0
4,1,0,1,0,0
...,...,...,...,...,...
71,0,1,0,0,1
72,0,1,0,0,1
73,0,1,0,0,1
74,0,1,0,0,1


In [47]:
both_params = np.linalg.pinv(x_gender_diet) @ y
both_params

array([2.29947098, 2.39455255, 0.9609117 , 0.92669313, 2.80641869])

Now, if we adjust for both `gender` and `diet` by removing the predicted values, we really do have means of very near 0 across `gender` and `diet`:

In [48]:
predicted = np.sum(both_params * np.array(x_gender_diet), axis=1)
y_both_adj = y - predicted
both_adj = diet_data.assign(both_adj=y_both_adj)
both_adj.groupby('gender')['both_adj'].mean()

gender
Female   -3.139607e-15
Male     -2.987509e-15
Name: both_adj, dtype: float64

In [49]:
both_adj.groupby('diet')['both_adj'].mean()

diet
A   -3.367677e-15
B   -2.700062e-15
C   -3.256654e-15
Name: both_adj, dtype: float64

## Extra sum of squares

We now have a way to calculate the remaining sum of squares for any combination of factors, or their interaction.

In [56]:
ss_diet = calc_sse_shortcut(y, x_diet)
ss_diet

410.40180740740743

In [57]:
ss_gender_diet = calc_sse_shortcut(y, x_gender_diet)
ss_gender_diet

410.2331115610072

Thus the Extra Sum of Squares when adding `gender` to the model is:

In [59]:
ess_gender = ss_diet - ss_gender_diet
ess_gender

0.16869584640022595

In [60]:
# Type II (2) sum of squares calculation ANOVA table.
sm.stats.anova_lm(unbal_fit, typ=2)

,sum_sq,df,F,PR(>F)
gender,0.168696,1.0,0.031379,0.859910
diet,60.417220,2.0,5.619026,0.005456
gender:diet,33.904068,2.0,3.153204,0.048842
Residual,376.329043,70.0,NaN,NaN


In [61]:
ss_gender = calc_sse_shortcut(y, x_gender)
ess_gender = ss_gender - ss_gender_diet
ess_gender

60.41721965816117

In [63]:
all_groups = diet_data['gender'].str.cat(diet_data['diet'], sep='-')
all_groups

0     Female-A
1     Female-A
2     Female-A
3     Female-A
4     Female-A
        ...   
71      Male-C
72      Male-C
73      Male-C
74      Male-C
75      Male-C
Name: gender, Length: 76, dtype: object

In [64]:
x_all_groups = pd.get_dummies(all_groups)
x_all_groups

,Female-A,Female-B,Female-C,Male-A,Male-B,Male-C
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
71,0,0,0,0,0,1
72,0,0,0,0,0,1
73,0,0,0,0,0,1
74,0,0,0,0,0,1


In [67]:
x_all = pd.concat([x_gender_diet, x_all_groups], axis=1)
x_all

,Female,Male,A,B,C,Female-A,Female-B,Female-C,Male-A,Male-B,Male-C
0,1,0,1,0,0,1,0,0,0,0,0
1,1,0,1,0,0,1,0,0,0,0,0
2,1,0,1,0,0,1,0,0,0,0,0
3,1,0,1,0,0,1,0,0,0,0,0
4,1,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
71,0,1,0,0,1,0,0,0,0,0,1
72,0,1,0,0,1,0,0,0,0,0,1
73,0,1,0,0,1,0,0,0,0,0,1
74,0,1,0,0,1,0,0,0,0,0,1


In [69]:
ss_all = calc_sse_shortcut(y, x_all)
ss_all

376.3290432900432

In [70]:
ess_inter = ss_gender_diet - ss_all
ess_inter

33.90406827096399

In [75]:
n_groups = len(all_groups.unique())
df_error = len(diet_data) - n_groups  # D.F. for remaining variation

We replicate all three F values in the ANOVA table above:

In [76]:
df_gender = len(diet_data['gender'].unique()) - 1  # 1
F_gender = (ess_gender / df_gender) / (ss_all / df_error)
F_gender

11.238052049072815

In [ ]:
```{python}
df_diet = len(adj_for_gd['diet'].unique()) - 1  # 2
F_diet = (diet_ss / df_diet) / (resid_ss / df_error)
F_diet
```

```{python}
# We have already adjusted the interaction values for gender and diet.
df_inter = n_groups - df_gender - df_diet - 1  # 2
F_inter = (inter_ss / df_inter) / (resid_ss / df_error)
F_inter
```